In [1]:
# DisulfideBond Playground
# Playing with the DisulfideBond class
# Author: Eric G. Suchanek, PhD.
# License: BSD
# Last Modification: 2/18/24
# Cα N, Cα, Cβ, C', Sγ Å °

import pandas as pd
import numpy as np

pd.set_option("display.width", 100)

import pyvista as pv
from pyvista import set_plot_theme

from proteusPy.data import *

from proteusPy import (
    Disulfide,
    DisulfideList,
    DisulfideLoader,
    DisulfideClass_Constructor,
    Load_PDB_SS,
    DisulfideClass_Constructor,
    generate_vector_dataframe,
)

from proteusPy.data import DATA_DIR

# pyvista setup for notebooks
pv.set_jupyter_backend("trame")
set_plot_theme("dark")

from plotly.subplots import make_subplots

In [ ]:
# Do this to reload the extracted disulfides and rebuild the loader, save it
# SUBSET = False
# PDB_SS = DisulfideLoader(verbose=True, subset=SUBSET, cutoff=8.0)
# PDB_SS.save(cutoff=8.0, subset=SUBSET)

In [2]:
PDB_SS = Load_PDB_SS(verbose=True, subset=False)

if PDB_SS is not None:
    PDB_SS.describe()

-> load_PDB_SS(): Reading /Users/egs/miniforge3/envs/ppydev/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
-> load_PDB_SS(): Done reading /Users/egs/miniforge3/envs/ppydev/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
    =========== RCSB Disulfide Database Summary ==============
       =========== Built: 2024-08-02 21:10:07 ==============
PDB IDs present:                    37183
Disulfides loaded:                  175803
Average structure resolution:       2.07 Å
Lowest Energy Disulfide:            2q7q_75D_140D
Highest Energy Disulfide:           6vxk_801B_806B
Cα distance cutoff:                 -1.00 Å
Total RAM Used:                     44.39 GB.
    ================= proteusPy: 0.97.0dev5 =======================


In [3]:
# 1mfe_137L_196L
ss1 = PDB_SS["4lb7_2D_30D"]  # -!*--
ss1.pprint()
ss2 = PDB_SS["2qhe_61A_91A"]
ss2.pprint()
ss3 = PDB_SS["3jad_214A_225A"]  # ++!++
ss3.pprint()

<Disulfide 4lb7_2D_30D, Source: 4lb7, Resolution: 1.0 Å 
Χ1-Χ5: -47.25°, -119.53°, 112.66°, -85.34° -51.07°, 175.57°, 5.60 kcal/mol 
Cα Distance: 4.47 Å 
Torsion length: 197.74 deg>
<Disulfide 2qhe_61A_91A, Source: 2qhe, Resolution: 2.0 Å 
Χ1-Χ5: -47.78°, -66.88°, -85.05°, -66.32° -75.83°, -46.07°, 1.66 kcal/mol 
Cα Distance: 5.70 Å 
Torsion length: 155.36 deg>
<Disulfide 3jad_214A_225A, Source: 3jad, Resolution: 3.0 Å 
Χ1-Χ5: 55.07°, 77.88°, -152.09°, 74.65° 59.39°, 102.75°, 6.76 kcal/mol 
Cα Distance: 5.09 Å 
Torsion length: 203.29 deg>


In [4]:
from proteusPy.utility import generate_vector_dataframe

trinary = generate_vector_dataframe(base=3)
trinary.head(64)

,chi1,chi2,chi3,chi4,chi5
0,-,-,-,-,-
1,-,-,-,-,+
2,-,-,-,-,*
3,-,-,-,+,-
4,-,-,-,+,+
...,...,...,...,...,...
59,-,*,-,+,*
60,-,*,-,*,-
61,-,*,-,*,+
62,-,*,-,*,*


In [5]:
quart = generate_vector_dataframe(base=4)
quart.head(128)

,chi1,chi2,chi3,chi4,chi5
0,-,-,-,-,-
1,-,-,-,-,+
2,-,-,-,-,*
3,-,-,-,-,@
4,-,-,-,+,-
...,...,...,...,...,...
123,-,+,@,*,@
124,-,+,@,@,-
125,-,+,@,@,+
126,-,+,@,@,*


In [6]:
all_tors_df = PDB_SS.TorsionDF
all_tors_df.describe()

,proximal,distal,chi1,chi2,chi3,chi4,chi5,energy,ca_distance,cb_distance,phi_prox,psi_prox,phi_dist,psi_dist,torsion_length,rho
count,175803.000000,175803.000000,175803.000000,175803.000000,175803.000000,175803.000000,175803.000000,175803.000000,175803.000000,175803.000000,175803.0,175803.0,175803.0,175803.0,175803.000000,175803.000000
mean,228.403685,276.162483,-45.290470,-5.704312,-4.211264,-23.018728,-30.414271,3.740453,5.721303,3.969993,-180.0,-180.0,-180.0,-180.0,232.944114,-26.704043
std,324.073651,327.718616,104.232793,108.923741,94.103268,111.353289,99.640017,2.498559,2.467271,2.672237,0.0,0.0,0.0,0.0,56.613700,107.651149
min,-86.000000,-62.000000,-179.999869,-179.999596,-179.981757,-179.999698,-179.999497,0.491737,0.000000,0.000000,-180.0,-180.0,-180.0,-180.0,77.516159,-179.997699
25%,42.000000,94.000000,-92.787497,-87.484379,-88.009322,-97.098004,-75.027621,1.970668,5.159570,3.688566,-180.0,-180.0,-180.0,-180.0,185.218153,-94.251849
50%,134.000000,188.000000,-64.312647,-54.602890,-65.048754,-66.352451,-59.686045,3.191229,5.714850,3.846219,-180.0,-180.0,-180.0,-180.0,231.988160,-59.410453
75%,293.000000,346.000000,-38.686988,96.004191,92.805726,82.462424,56.376156,4.599670,6.269686,4.025796,-180.0,-180.0,-180.0,-180.0,272.324313,53.762581
max,8025.000000,9004.000000,179.999973,179.998744,179.994824,179.999995,179.999702,19.426512,205.843810,337.252323,-180.0,-180.0,-180.0,-180.0,383.768927,179.998397


In [7]:
import time

start = time.time()

# pyvista setup for notebooks
pv.set_jupyter_backend("trame")
set_plot_theme("dark")

PDB_SS = Load_PDB_SS(verbose=True, subset=False)

_PBAR_COLS = 80


def analyze_classes(loader: DisulfideLoader) -> DisulfideList:
    classes = loader.classdict
    tot_classes = len(classes)
    res_list = DisulfideList([], "SS_Class_Avg_SS")

    pbar = enumerate(classes)
    for idx, cls in pbar:
        print(f"--> analyze_classes(): {cls} {idx+1}/{tot_classes}")
        class_ss_list = loader.from_class(cls)
        fname = f"./classes/ss_class_{idx}.png"
        class_ss_list.TorsionGraph(display=False, save=True, fname=fname)
        fname2 = f"./classes/ss_consensus_{idx}.png"
        avg_conformation = class_ss_list.Average_Conformation
        ssname = f"{cls}_avg"
        exemplar = Disulfide(ssname)
        exemplar.build_model(
            avg_conformation[0],
            avg_conformation[1],
            avg_conformation[2],
            avg_conformation[3],
            avg_conformation[4],
        )
        res_list.append(exemplar)
    return res_list


def build_consensus_list(loader: DisulfideLoader) -> DisulfideList:
    classes = loader.classdict
    tot_classes = len(classes)
    res_list = DisulfideList([], "SS_Classes")

    pbar = enumerate(classes)
    for idx, cls in pbar:
        print(f"--> build_consensus_list(): {cls} {idx+1}/{tot_classes}")
        class_ss_list = loader.from_class(cls)
        fname2 = f"./classes/ss_consensus_{idx}.png"

        # compute and build a consensus 'average' conformation
        avg_conformation = class_ss_list.Average_Conformation
        ssname = f"{cls}_avg"
        exemplar = Disulfide(ssname)
        exemplar.build_model(
            avg_conformation[0],
            avg_conformation[1],
            avg_conformation[2],
            avg_conformation[3],
            avg_conformation[4],
        )
        res_list.append(exemplar)

    return res_list


avg_ss_list = DisulfideList([], "SS_Class_Consensus")
avg_ss_list = analyze_classes(PDB_SS)


end = time.time()
elapsed = end - start

-> load_PDB_SS(): Reading /Users/egs/miniforge3/envs/ppydev/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 
-> load_PDB_SS(): Done reading /Users/egs/miniforge3/envs/ppydev/lib/python3.11/site-packages/proteusPy/data/PDB_SS_ALL_LOADER.pkl... 


AttributeError: 'DisulfideLoader' object has no attribute 'classdict'

In [ ]:
avg_ss_list.display(style="sb", light=True)

In [ ]:
tors_df = PDB_SS.getTorsions()
tors_df.describe()

In [ ]:
ss1 = Disulfide("bogus")
ss1.build_model(-60, -60, -84, -60, -60)
ss1.pprint()

In [ ]:
PDB_SS["2q7q_75D_140D"]

In [ ]:
PDB_SS2 = Load_PDB_SS(subset=True, verbose=True)
# PDB_SS.describe()

In [1]:
import py3Dmol
import urllib.request

import py3Dmol
import urllib.request


def Odisplay_pdb_structure():
    # download PDB structure 3q7q
    pdb_url = "https://files.rcsb.org/download/2q7q.pdb"
    pdb_data = urllib.request.urlopen(pdb_url).read().decode("utf-8")

    # create viewer object
    viewer = py3Dmol.view(width=1024, height=1024)

    # add PDB data to viewer
    viewer.addModel(pdb_data, "pdb")

    # select residues 75 and 150 from chain 'D'
    sel_str = ":D and (resi 75 or resi 140)"
    # viewer.setStyle({'resi': [75, 140], 'chain': 'D'}, {'stick': {'color': 'green', 'radius': 0.6}, 'sphere': {'color': 'green', 'radius': 1}}, sel_str)
    viewer.setStyle(sel_str, {"sphere": {"scale": 0.5}, "color": "element"})

    # display all other residues as a ribbon colored by chain
    viewer.setStyle({}, {"cartoon": {"color": "spectrum"}})

    # center and zoom to selection
    viewer.zoomTo({"sel": sel_str})

    # show viewer
    return viewer.show()


import urllib.request
import py3Dmol


def oodisplay_pdb_structure():
    # Download PDB structure 2q7q
    pdb_url = "https://files.rcsb.org/download/2q7q.pdb"
    pdb_file = urllib.request.urlopen(pdb_url).read().decode("utf-8")

    # Initialize viewer
    view = py3Dmol.view(width=800, height=600)

    # Select residues 75 and 140 from chain D
    selection = "(resid 81 or resid 113)"

    # Add selected atoms as spheres scaled by atom radius and colored by atom type
    view.addModel(pdb_file, "pdb")
    view.setStyle(selection, {"sphere": {"scale": 0.5}, "color": "element"})

    # Display all other residues as a ribbon colored by secondary structure
    view.setStyle("not " + selection, {"cartoon": {"color": "spectrum"}})

    # Set camera and render the viewer
    view.zoomTo(selection)
    view.render()


import py3Dmol
import requests


def display_pdb_structure():
    # Download PDB structure 2q7q
    pdb_id = "2q7q"
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    pdb_file = requests.get(pdb_url).text

    # Initialize viewer
    view = py3Dmol.view(width=800, height=600)

    # Select residues 75 and 140 from chain D
    selection = {"chain": "D", "resi": "75, 140"}

    # Add selected atoms as spheres scaled by atom radius and colored by atom type
    view.addModel(pdb_file, "pdb")
    view.setStyle(selection, {"sphere": {"scale": 0.5}, "color": "element"})

    # Display all other residues as a ribbon colored by secondary structure
    view.setStyle({}, {"cartoon": {"color": "spectrum"}})

    # Set camera and render the viewer
    view.zoomTo(selection)
    view.render()


import requests
import py3Dmol


def odisplay_cys_residues():
    # Download PDB structure 2q7q
    pdb_id = "2q7q"
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    pdb_file = requests.get(pdb_url).text

    # Initialize viewer
    view = py3Dmol.view(width=800, height=800)

    # Select all Cys residues
    selection = {"resn": "CYS"}

    # Add selected atoms as spheres scaled by atom radius and colored green
    view.addModel(pdb_file, "pdb")
    view.setStyle(selection, {"sphere": {"scale": 0.5}, "color": "green"})

    # Set camera and render the viewer
    view.zoomTo(selection)
    view.render()


import requests
import py3Dmol


def display_cys_residues():
    # Download PDB structure 2q7q
    pdb_id = "2q7q"
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    pdb_file = requests.get(pdb_url).text

    # Initialize viewer
    view = py3Dmol.view(width=800, height=600)

    # Add protein backbone as ribbon
    view.addModel(pdb_file, "pdb")
    view.setStyle({"cartoon": {"color": "spectrum"}})

    # Select all Cys residues and display as green spheres
    selection = {"resn": "CYS"}
    view.addStyle(selection, {"sphere": {"color": "green", "radius": 0.3}})

    # Set camera and render the viewer
    view.zoomTo(selection)
    view.render()


import requests
import py3Dmol


def display_cys_and_ss_bonds():
    # Download PDB structure 2q7q
    pdb_id = "2q7q"
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    pdb_file = requests.get(pdb_url).text

    # Initialize viewer
    view = py3Dmol.view(width=800, height=800)

    # Add protein as sticks
    view.addModel(pdb_file, "pdb")
    # view.setStyle({'stick': {}})

    # Select all Cys residues and SSBOND bonds and display as green sticks
    cys_selection = {"resn": "CYS"}
    ssbond_selection = {"ss": "disulf"}
    view.addStyle(
        cys_selection, {"stick": {"colorscheme": "greenCarbon", "radius": 0.2}}
    )
    view.addStyle(
        ssbond_selection,
        {"stick": {"colorscheme": "greenCarbon", "radius": 1.0}},
    )

    # Set camera and render the viewer
    # view.zoomTo(cys_selection)
    view.render()


import requests
import py3Dmol


def display_disulfide_bonds():
    # Download PDB structure 2q7q
    pdb_id = "2q7q"
    pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    pdb_file = requests.get(pdb_url).text

    # Initialize viewer
    view = py3Dmol.view(width=800, height=600)

    # Add protein as sticks
    view.addModel(pdb_file, "pdb")
    view.setStyle({"stick": {}})

    # Select all disulfide bonds and display as green sticks
    ssbond_selection = {"ss": "disulf"}
    view.addStyle(
        ssbond_selection,
        {"stick": {"colorscheme": "greenCarbon", "radius": 0.2}},
    )

    # Set camera and render the viewer
    view.zoomTo(ssbond_selection)
    view.render()


display_disulfide_bonds()

ModuleNotFoundError: No module named 'py3Dmol'

In [ ]:
import py3Dmol

viewer = py3Dmol.view(query="pdb:2q7q")
viewer.setStyle({"cartoon": {}})

viewer.setStyle(
    {"chain": "D", "resn": "CYS"},
    {"stick": {"colorscheme": "lightgreenCarbon"}},
)
# viewer.addLabel('Chain D', {'fontColor':'orange', 'backgroundColor':'lightgray'}, {'chain': ['D']})
ssbond_selection = {"ss": "disulf"}
viewer.addStyle(
    ssbond_selection, {"stick": {"colorscheme": "greenCarbon", "radius": 0.2}}
)

# viewer.setStyle({'resn': 'CYS'},{'stick': {'colorscheme': 'orangeCarbon'}})

viewer.show()

In [ ]:
sslist = PDB_SS["4yys"]
sslist

In [ ]:
sslist = PDB_SS.SSList
LHS_neighbors = sslist.nearest_neighbors(-60, -60, -90, -60, -60, 20.0)
LHS_neighbors.length
# modelss = Disulfide('model', proximal=1, distal=2, pdb_id='EGS')
# modelss.build_model(-60,-60, -90, -60, -60)

In [ ]:
# We use the entire database contained in PDB_SS.SSList for our search here.
# The entire database can be scanned in 13 seconds on the M1 Pro Macbook Pro.

ssmin_enrg, ssmax_enrg = PDB_SS.SSList.minmax_energy
ssmin_enrg.pprint()
ssmax_enrg.pprint()
minmax = DisulfideList([ssmin_enrg, ssmax_enrg], "minmax")
# minmax.display()

In [ ]:
ssmin_enrg.Torsion_Distance(ssmax_enrg)

In [ ]:
ssmin_enrg.Torsion_Distance(ssmin_enrg)

In [ ]:
import proteusPy
from proteusPy.DisulfideLoader import Load_PDB_SS
from proteusPy.DisulfideList import DisulfideList
from proteusPy.Disulfide import Disulfide

PDB_SS = None
PDB_SS = Load_PDB_SS(verbose=True, subset=False)

ss_list = DisulfideList([], "tmp")
sslist = PDB_SS.SSList
ssmin_enrg, ssmax_enrg = PDB_SS.SSList.minmax_energy

low_energy_neighbors = DisulfideList([], "Neighbors")
low_energy_neighbors = ssmin_enrg.Torsion_neighbors(sslist, 5)


low_energy_neighbors.display_overlay()

In [5]:
tot = low_energy_neighbors.length
drms = low_energy_neighbors.Average_Distance
trms = low_energy_neighbors.Average_Torsion_Distance
erms = low_energy_neighbors.Average_Energy

print(
    f"Low Energy Neighbors: {tot}, Distance: {drms:.2f} Torsions: {trms:.2f}, Energy: {erms:.2f}"
)

Low Energy Neighbors: 25, Distance: 0.85 Torsions: 5.55, Energy: 0.53


In [ ]:
low_energy_neighbors.display_overlay()

In [ ]:
high_energy_neighbors = DisulfideList([], "neighbors2")
high_energy_neighbors = ssmax_enrg.Torsion_neighbors(sslist, 20)

tot2 = high_energy_neighbors.length()
print(f"Neighbors: {tot2}")

In [ ]:
drms = high_energy_neighbors.Average_Distance()
trms = high_energy_neighbors.Avg_Torsion_Distance()
erms = high_energy_neighbors.Average_Energy()

print(
    f"High Energy Neighbors: {tot}, Distance: {drms:.2f} Torsions: {trms:.2f}, Energy: {erms:.2f}"
)

In [ ]:
high_energy_neighbors.display_overlay()

In [ ]:
ssmin = Disulfide()
ssmax = Disulfide()

ssmin, ssmax = sslist.minmax_distance()
ssmin.pprint()
ssmax.pprint()

minmax = DisulfideList([ssmin, ssmax], "minmax")
minmax.display()

In [ ]:
# one disulfide from the database
ss = Disulfide()
ss = PDB_SS[0]
# ss.pprint_all()

# get all disulfides for one structure. Make a
# DisulfideList object to hold it
ss4yys = DisulfideList([], "4yys")
ss4yys = PDB_SS["4yys"]

# ss4crn = DisulfideList([], '1crn')
# ss4crn = PDB_SS['1crn']

tot_ss = len(ss4yys)  # number off ssbonds
print(f"tot {tot_ss}")

In [ ]:
print(f"{ss4yys.Torsion_Distance()}")

In [ ]:
ca_df = sslist.distance_df
ca_df.head(10)

In [ ]:
ss6fuf = PDB_SS["6fuf"]

In [ ]:
ss1 = ss4yys.get_by_name("4yys_22A_65A")
ss1

In [ ]:
ss4yys_a = ss4yys.by_chain("A")
ss4yys_a.Torsion_Distance()

In [ ]:
ss4yys_b = ss4yys.by_chain("B")
ss4yys_b.Torsion_Distance()

In [ ]:
ss4yys_a1 = ss4yys_a[0]
# print(ss4yys_a1.repr_ss_coords())

In [ ]:
ss4yys_b1 = ss4yys_b[0]
ss4yys_b1

In [ ]:
ss4yys_a1.Distance_RMS(ss4yys_b1)

In [ ]:
ss4yys_a1.Torsion_Distance(ss4yys_b1)

In [ ]:
ss4yys_a1 == ss4yys_b1

In [ ]:
chns = ss4yys.get_chains()
ss4yys.has_chain("yyy")
chns

In [ ]:
# load SS bonds by PDB ID
ss1 = PDB_SS["4yys"]
print(ss1)
print(ss1[0].get_full_id())

In [ ]:
# you can loop over the IDList list and extract by ID
#
for id in PDB_SS.IDList[:5]:  # just show last 5
    # get the SS bonds for the given ID
    ssb = PDB_SS[id]
    numb_ss = len(ssb)
    print(f"ID: {id} has {numb_ss} Disulfides:")
    for bond in ssb:
        print(bond)
    print("\n")

In [ ]:
ss_list = PDB_SS.getlist()
ss0 = ss_list[0]

print(ss0.proximal_residue_fullid)
print(ss0.chi3)
len(ss_list)

In [ ]:
# routine creates 2 lists  for left-handed and right-handed disulfides
ss_list = PDB_SS.getlist()
left_handed = DisulfideList([], "left_handed")
right_handed = DisulfideList([], "right_handed")

i = 0

for i in range(len(ss_list)):
    ss = ss_list[i]
    if ss.chi3 < 0:
        left_handed.append(ss)
    else:
        right_handed.append(ss)


print(f"Left Handed: {len(left_handed)}, Right Handed: {len(right_handed)}")

In [ ]:
from proteusPy.Disulfide import Disulfide

# make some empty disulfides
ss1 = Disulfide("ss1")
ss2 = Disulfide("ss2")

# make a DisulfideList containing ss1, named 'tmp'
sslist = DisulfideList([ss1], "tmp")
sslist.append(ss2)

# load the PDB Disulfide database
PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True, subset=True)

# extract a disulfide with typical index
ss1 = PDB_SS[0]
# print(f'{ss1.pprint_all()}')

# grab a subset via slicing
subset = DisulfideList(PDB_SS[0:10], "subset")

In [ ]:
rms = subset.Torsion_Distance()
rms

In [ ]:
torsions = PDB_SS.getTorsions()
torsions.sort_values(by=["energy"], ascending=False, inplace=True)

torsions.head(10)

In [ ]:
toget = 200

badlist = torsions["ss_id"]

bad_SS_list = DisulfideList([], "20 top high energy")
ss = Disulfide()

for i in range(toget):
    ssid = torsions.iloc[i]["ss_id"]
    ss = PDB_SS.get_by_name(ssid)
    # print(f'ID: {ss.name}: {ss.energy:.2f} kcal/mol CA: {ss.ca_distance:.2f}')
    bad_SS_list.append(ss)

In [ ]:
tors_bad_rms = bad_SS_list.Torsion_Distance()
tors_bad_rms

In [ ]:
goodlist = torsions["ss_id"]

good_SS_list = DisulfideList([], "20 top low energy")
ss = Disulfide()

for i in range(toget):
    if i == 0:
        continue

    ssid = torsions.iloc[-i]["ss_id"]
    ss = PDB_SS.get_by_name(ssid)
    # print(f'ID: {ss.name}: {ss.energy:.2f} kcal/mol CA: {ss.ca_distance:.2f}')
    good_SS_list.append(ss)

In [ ]:
tors_good_rms = good_SS_list.Torsion_Distance()
tors_good_rms

In [ ]:
from scipy.spatial import distance_matrix

good_array = good_SS_list.get_torsion_array()
bad_array = bad_SS_list.get_torsion_array()
good_array.shape

In [ ]:
dm1 = distance_matrix(good_array, good_array)
dm2 = distance_matrix(bad_array, bad_array)

In [ ]:
import numpy as np


def EDM(A, B):
    p1 = np.sum(A**2, axis=1)[:, np.newaxis]
    p2 = np.sum(B**2, axis=1)
    p3 = -2 * np.dot(A, B.T)
    res = p1 + p2 + p3
    # res = numpy.sqrt(res)
    return res

In [ ]:
dm3 = np.sqrt(dm2)

In [ ]:
import plotly_express as px

fig = px.imshow(dm1)
fig.show()

In [ ]:
import plotly_express as px

fig = px.imshow(dm2)
fig.show()

In [ ]:
dm1.shape
dm1

In [ ]:
df = pd.DataFrame(dm1.copy())
df.describe()

In [ ]:
x = np.arange(0, 20, 20)
y = np.arange(0, 20, 20)
z = dm1
x, y = np.meshgrid(x, y)

In [ ]:
# Create and plot structured grid
grid = pv.StructuredGrid(x, y, z)
grid.plot()

In [ ]:
def doit(tot_ss):
    brad = BOND_RADIUS * (1 - ((tot_ss / 10)) / 50)
    return brad

In [ ]:
doit(51)

### Disulfide Families

In [ ]:
import proteusPy
from proteusPy import DisulfideLoader
from proteusPy.Disulfide import Disulfide
from proteusPy.DisulfideList import DisulfideList

PDB_SS = DisulfideLoader(verbose=True, subset=True, datadir="/Users/egs/pdb/data/")

In [ ]:
PDB_SS.SSDict

In [ ]:
PDB_SS["4yys"]

In [ ]:
# retrieve the torsions dataframe
from proteusPy.Disulfide import Torsion_DF_Cols

_SSdf = PDB_SS.getTorsions()
_near = _SSdf["ca_distance"] < 9.0
# there are a few structures with bad SSBonds. Their
# CA distances are > 7.0. We remove them from consideration
# below

SS_df = _SSdf[_near]
SS_df = SS_df[Torsion_DF_Cols].copy()


_cols = ["chi1", "chi2", "chi3", "chi4", "chi5"]
X = SS_df[_cols].copy()
tot = X.shape[0]

tors = X.to_numpy()

In [ ]:
# chat gpt generated
import numpy as np
from sklearn.neighbors import KDTree


def nearest_neighbors(samples):
    # Sample the 5-dimensional vector space

    # Build a KDTree from the samples
    tree = KDTree(samples, leafsize=40)

    # Find the nearest neighbors for each sample
    distances, indices = tree.query(samples, k=5)

    # The first nearest neighbor of each sample is the sample itself, so we can ignore that
    nearest_neighbors = indices[:, 1]
    distances = distances[:, 1]

    return nearest_neighbors, distances


def nearest_neighbors_radial(samples, radius):
    """
    Find the neighbors for the given array of torsions using KDTree.

    :param samples: Neighbor array
    :param radius: Search radius (degrees)
    :return: List of indices for neighbors within the cutoff
    """

    # Build a KDTree from the samples
    tree = KDTree(samples, leaf_size=40)

    # Find the nearest neighbors for each sample
    indices = tree.query_radius(samples, r=radius)

    # The first nearest neighbor of each sample is the sample itself, so we can ignore that
    # nearest_neighbors = indices[:, 1]
    nearest_neighbors = indices
    return nearest_neighbors


def build_SSlist(loader: DisulfideLoader, indices) -> DisulfideList:
    """
    For a given ```DisulfideLoader``` and list of Disulfide indices return the Disulfides in a DisulfideList

    :param indices: List of SS by index
    :return: DisulfideList of SS
    """
    res = DisulfideList([], "tmp")
    sslist = loader.SSList

    tot = len(indices)
    res = DisulfideList([sslist[indices[i]] for i in range(tot)], "tmp")
    return res

In [ ]:
#

_cols = ["chi1", "chi2", "chi3", "chi4", "chi5"]
tor_df = SS_df[_cols].copy()

X = tor_df.copy()
tot = X.shape[0]

tors = X.to_numpy()

nneighbors = nearest_neighbors_radial(X, radius=20)
print(f"{len(nneighbors)}")

In [ ]:
from tqdm import tqdm

# for each SS, build a list of neighbors
neighbor_list = []

pbar = tqdm(range(nneighbors.shape[0]))

for i in pbar:
    neigh = nneighbors[i]
    res = build_SSlist(PDB_SS, neigh)
    neighbor_list.append(res)

In [ ]:
# for the neighbor list,

In [ ]:
import numpy as np
import sklearn
from sklearn.neighbors import KDTree

tree = KDTree(X, leaf_size=2)
dist, ind = tree.query(X, k=5)
print(ind)  # indices of 3 closest neighbors

print(dist)  # distances to 3 closest neighbors

In [ ]:
from sklearn.neighbors import KDTree

tree = KDTree(X, leaf_size=3)
print(tree.query_radius(X[:1], r=10, count_only=True))

ind = tree.query_radius(X[:1], r=10)
print(ind)  # indices of neighbors within distance 0.3

In [ ]:
import numpy as np

rng = np.random.RandomState(42)
tree = KDTree(X)
tree.kernel_density(X[:5], h=0.5, kernel="gaussian")

In [ ]:
import pandas as pd

df = pd.read_csv("classes.csv")
with open("classes.md", "w") as md:
    df.to_markdown(buf=md, tablefmt="grid")

In [ ]:
help(fig.update_traces)

In [ ]:
import proteusPy
from proteusPy.utility import image_to_ascii_art

image_to_ascii_art("logo_flat.png", 120)

In [ ]:
proteusPy.__version__